In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Reading Data

In [3]:
df = pd.read_csv('../../Dataset/NSL_new.csv')

In [4]:
df = df.drop( df[ (df.label != "normal") & (df.label != "neptune") & (df.label != "back") & (df.label != "land") & (df.label != "pod") & (df.label != "smurf") & (df.label != "teardrop") & (df.label != "teardrop") & (df.label != "mailbomb") & (df.label != "apache2") & (df.label != "processtable") & (df.label != "udpstorm") & (df.label != "worm")].index )
print(df.shape)

(113270, 123)


In [5]:
df = df.sample(n=20000)

In [6]:
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label
46161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,neptune
54568,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,neptune
25408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neptune
105343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,neptune
69624,0,209,171,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal


In [7]:
X = df.drop(['label'], axis=1)
y = df['label']

In [8]:
scoring_metrics = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average="micro"),
           'recall' : make_scorer(recall_score, average="micro"), 
           'f1_score' : make_scorer(f1_score, average="micro")}

In [9]:
y = y.replace( to_replace =  'normal' , value = 0 )

y = y.replace( to_replace =  'neptune' , value = 1 )
y = y.replace( to_replace =  'back' , value = 1 )
y = y.replace( to_replace =  'land' , value = 1 )
y = y.replace( to_replace =  'pod' , value = 1 )
y = y.replace( to_replace =  'smurf' , value = 1 )
y = y.replace( to_replace =  'teardrop' , value = 1 )
y = y.replace( to_replace =  'mailbomb' , value = 1 )
y = y.replace( to_replace =  'apache2' , value = 1 )
y = y.replace( to_replace =  'processtable' , value = 1 )
y = y.replace( to_replace =  'udpstorm' , value = 1 )
y = y.replace( to_replace =  'worm' , value = 1 )

 # Chi Squared

In [10]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [11]:
print("before transform:",X)
selector=SelectKBest(score_func=chi2,k=4)
fit = selector.fit(X,y)
features = fit.transform(X)
print("scores_:",fit.scores_)
print("pvalues_:",fit.pvalues_)
print("selected index:",fit.get_support(True))
print("after transform:",fit.transform(X)) 
X = fit.transform(X)

before transform:         duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
46161          0          0          0     0               0       0    0   
54568          0          0          0     0               0       0    0   
25408          0          0          0     0               0       0    0   
105343         0          0          0     0               0       0    0   
69624          0        209        171     0               0       0    0   
...          ...        ...        ...   ...             ...     ...  ...   
92103          0        226        486     0               0       0    0   
73389          0       1032          0     0               0       0    0   
101963         0         42        127     0               0       0    0   
115765        20      11773       1459     0               0       0    0   
54763          0        387        716     0               0       0    0   

        num_failed_logins  logged_in  num_compromised  ..

In [12]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# (KNN-RF-ADA)_(DT-MLP)_MLP

In [13]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1)),
                        ('ada_1', AdaBoostClassifier(random_state=0, learning_rate=0.1, n_estimators=1000))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                       ]

In [14]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=MLPClassifier())

In [15]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

  Accuracy Precision   Recall
0   0.9928  0.993593  0.98872


In [16]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(KNN-RF-ADA)_(DT-MLP)_MLP_ChiSquared.csv')

# (KNN-RF)_(DT-MLP)_MLP

In [17]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=MLPClassifier())

In [18]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

  Accuracy Precision    Recall
0   0.9932  0.992145  0.991172


In [19]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(KNN-RF)_(DT-MLP)_MLP_ChiSquared.csv')

# (RF-MLP)_(DT-KNN)_MLP

In [20]:
# Create Learners per layer
layer_one_estimators = [
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, min_samples_split=0.1)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1))
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=MLPClassifier())

In [21]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  Accuracy Precision    Recall
0   0.9892   0.98964  0.983816


In [22]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(RF-MLP)_(DT-KNN)_MLP_ChiSquared.csv')

# (KNN-RF)_(DT-MLP)_DT

In [23]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                       ]

In [24]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=DecisionTreeClassifier())

In [25]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

  Accuracy Precision    Recall
0   0.9884  0.992541  0.978911


In [26]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(KNN-RF)_(DT-MLP)_DT_ChiSquared.csv')

# (ADA-MLP)_(DT-RF)_MLP

In [27]:
# Create Learners per layer
layer_one_estimators = [
                        ('ada_1', AdaBoostClassifier(random_state=0, learning_rate=0.1, n_estimators=1000)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=MLPClassifier())

In [28]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  Accuracy Precision    Recall
0   0.9934  0.992149  0.991663


In [29]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(ADA-MLP)_(DT-RF)_MLP_ChiSquared.csv')

# (MLP-RF)_(DT-KNN)_DT

In [30]:
# Create Learners per layer
layer_one_estimators = [
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20))
                       ]

layer_two_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10))
                       ]

In [31]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=DecisionTreeClassifier())

In [32]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  Accuracy Precision    Recall
0   0.9888  0.988664  0.983816


In [33]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(MLP-RF)_(DT-KNN)_DT_ChiSquared.csv')

# (KNN-RF)_(DT-MLP)_KNN

In [34]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                       ]

In [35]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=KNeighborsClassifier())

In [36]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

  Accuracy Precision    Recall
0   0.9934  0.991667  0.992153


In [37]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(KNN-RF)_(DT-MLP)_KNN_ChiSquared.csv')

# (MLP-RF)_(DT-KNN)_KNN

In [38]:
# Create Learners per layer
layer_one_estimators = [
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1))
                       ]

In [39]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=KNeighborsClassifier())

In [40]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  Accuracy Precision    Recall
0   0.9886   0.99108  0.980873


In [41]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(MLP-RF)_(DT-KNN)_KNN_ChiSquared.csv')

# (KNN-MLP)_(DT-RF)_KNN

In [42]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1))
                       ]

In [43]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=KNeighborsClassifier())

In [44]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acurcy = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recal = recall_score(y_test, y_pred)

clf_Score = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])
clf_Score.at[0, 'Accuracy'] = acurcy
clf_Score.at[0, 'Precision'] = prec
clf_Score.at[0, 'Recall'] = recal
print(clf_Score)

  Accuracy Precision    Recall
0   0.9904  0.989189  0.987249


In [45]:
clf_Score.to_csv('NSL_DOS_Stacking_level2_(KNN-MLP)_(DT-RF)_KNN_ChiSquared.csv')